In [1]:
# Required packages:
library(SCopeLoomR)
library(AUCell)
library(SCENIC)

# For some of the plots:
#library(dplyr)
library(KernSmooth)
library(RColorBrewer)
library(plotly)
library(BiocParallel)
library(grid)
library(ComplexHeatmap)
library(data.table)
library(gprofiler2)
library(stringr)
library(dplyr)
library(tibble)
library(parallel)


Attachement du package : ‘SCopeLoomR’


L'objet suivant est masqué depuis ‘package:base’:

    flush



Attachement du package : ‘SCENIC’


L'objet suivant est masqué depuis ‘package:AUCell’:

    plotEmb_rgb


KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

Le chargement a nécessité le package : ggplot2


Attachement du package : ‘plotly’


L'objet suivant est masqué depuis ‘package:ggplot2’:

    last_plot


L'objet suivant est masqué depuis ‘package:stats’:

    filter


L'objet suivant est masqué depuis ‘package:graphics’:

    layout


ComplexHeatmap version 2.10.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly exp

In [2]:
getwd()

[1] "/work/adufour/notebook/scenic/visualisation_resultats"

In [3]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [5]:
setwd("/home/adufour/work/scenic/scenic_pig/direct_annot_v6/out")

In [10]:
scenicLoomPath <- file.path("/home/adufour/work/scenic/scenic_pig/direct_annot_v6/out/scenic/10x_PBMC/scenic_visualize.loom")
motifEnrichmentFile <- file.path("/home/adufour/work/scenic/scenic_pig/direct_annot_v6/out/scenic/10x_PBMC/multi_runs_cistarget/multi_runs_features_mtf.csv.gz")
scenicLoomPath2 <- file.path("/home/adufour/work/scenic/scenic_pig/pigs.loom")
file.exists(scenicLoomPath)
file.exists(motifEnrichmentFile)

[1] TRUE

[1] TRUE

In [11]:
loom <- open_loom(scenicLoomPath)
    # Read information from loom file:
    exprMat <- get_dgem(loom)
        exprMat_log <- log2(exprMat+1) # Better if it is logged/normalized
    regulons_incidMat <- get_regulons(loom, column.attr.name="Regulons")
        regulons <- regulonsToGeneLists(regulons_incidMat)
    regulonAUC <- get_regulons_AUC(loom, column.attr.name='RegulonsAUC')
    regulonAucThresholds <- get_regulon_thresholds(loom)
        regulonAucThresholds <- setNames(as.numeric(names(regulonAucThresholds)), regulonAucThresholds)
    embeddings <- get_embeddings(loom)
close_loom(loom)

In [12]:
cellInfo <- read.csv(file = '/home/adufour/work/notebook/scenic/preparation/cell_type.csv')

In [13]:
motifEnrichment <- data.table::fread(motifEnrichmentFile, header=T, skip=1)[-3,]
colnames(motifEnrichment)[1:2] <- c("TF", "MotifID")
motifEnrichment = motifEnrichment[-1,]

In [14]:
convertmotifench <- gconvert(
motifEnrichment$TF,
organism = "sscrofa",
numeric_ns = "",
mthreshold = Inf,
filter_na = TRUE
)

In [15]:
motifEnrichment$TF <- convertmotifench$name

In [16]:
regulon_name <- str_split(rownames(regulonAUC), "_", simplify = TRUE)

In [17]:
regulon_name_converted <- gconvert(
regulon_name[,1],
organism = "sscrofa",
numeric_ns = "",
mthreshold = Inf,
filter_na = TRUE
)

In [18]:
regulon_name_converted$concatenate <- paste(regulon_name_converted$name, regulon_name[,2])

In [19]:
regulonAUC@NAMES <- regulon_name_converted$concatenate

treshold

In [20]:
regulon_name_thre <- str_split(names(regulonAucThresholds), "_", simplify = TRUE)

In [21]:
regulon_name_converted_thre <- gconvert(
regulon_name_thre[,1],
organism = "sscrofa",
numeric_ns = "",
mthreshold = Inf,
filter_na = TRUE
)

In [22]:
regulon_name_converted_thre$concatenate <- paste(regulon_name_converted_thre$name, regulon_name_thre[,2])

In [23]:
names(regulonAucThresholds) <- regulon_name_converted_thre$concatenate

In [24]:
rownames(regulons_incidMat) <- regulon_name_converted_thre$concatenate

In [25]:
dataframeincid <- gconvert(
colnames(regulons_incidMat),
organism = "sscrofa",
numeric_ns = "",
mthreshold = Inf,
filter_na = FALSE
)

In [27]:
colnames(regulons_incidMat) <- dataframeincid$name

In [28]:
targetconverter <- function(x) {
    convertmotifench <- gconvert(
    x,
    organism = "sscrofa",
    numeric_ns = "",
    mthreshold = Inf,
    filter_na = TRUE
    )
    return(convertmotifench$name)
}

In [29]:
names(regulons) <- regulon_name_converted_thre$concatenate

In [30]:
regulonsbis <- regulons

In [31]:
regulons <-mclapply(regulons, targetconverter,  mc.cores = 10)

In [32]:
save.image("/home/adufour/work/rds_storage/SCENIC/scenic_converted.RData")